In [ ]:
%matplotlib inline
import os

import cmasher as cmr
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.ndimage
import scipy.signal

folder = r"J:\ctgroup\Edward\DATA\VMI\20250123\Propylene Oxide 3W"
folder = r"J:\ctgroup\Edward\DATA\VMI\20250122\find_linear"
folder = r"J:\ctgroup\Edward\DATA\VMI\20250121\xe_ellipticity"
files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".mat")]
angles = [float(f.split("\\")[-1].split("_")[0]) for f in files]

angles, files = zip(*sorted(zip(angles, files)))


def load_file(file, smoothed=True):
    data = scipy.io.loadmat(file, squeeze_me=True)
    px, py, pz = data["px"], data["py"], data["pz"]
    h = np.histogram(np.sqrt(px ** 2 + py ** 2 + pz ** 2), bins=500, range=(0, 1))

    hist, edges = h[0], h[1]
    if smoothed:
        hist = scipy.signal.savgol_filter(hist, 51, 3)
    hist = hist / np.max(hist)

    peaks, properties = scipy.signal.find_peaks(hist, prominence=0.07)
    peak_widths = scipy.signal.peak_widths(hist, peaks, rel_height=0.2)[0]

    ring_vals = np.array([edges[int(peaks[i])] for i in range(len(peaks))])
    ring_widths = np.array([edges[int(peak_widths[i])] for i in range(len(peaks))])
    #
    # plt.figure(file)
    # plt.plot(edges[:-1],hist)
    # plt.plot(edges[peaks],hist[peaks],'x')
    # [plt.plot([ring_vals[i]-ring_widths[i]*2,ring_vals[i]+ring_widths[i]*2],[0.5,0.5]) for i in range(len(ring_vals))]

    pr = np.sqrt(px ** 2 + py ** 2 + pz ** 2)
    masks = [
        (pr > ring_vals[i] - ring_widths[i] * 2)
        & (pr < ring_vals[i] + ring_widths[i] * 2)
        for i in range(len(ring_vals))
    ]

    rings = [(px[masks[i]], py[masks[i]], pz[masks[i]]) for i in range(len(ring_vals))]
    return {
        "px": px,
        "py": py,
        "pz": pz,
        "rings": rings,
        "ring_vals": ring_vals,
        "ring_widths": ring_widths,
    }


data = [load_file(f) for f in files]

In [ ]:
n = 256
thetav, phiv = np.linspace(0, np.pi, n), np.linspace(-np.pi, np.pi, n * 2)
theta, phi = np.meshgrid(thetav, phiv)
dtheta, dphi = np.pi / n, 2 * np.pi / n / 2
domega = np.sin(theta) * dtheta * dphi
lmax = 19
Y = {
    (l, m): scipy.special.sph_harm(m, l, phi, theta)
    for l in range(lmax + 1)
    for m in range(-l, l + 1)
}
Y_real = {
    (l, m): np.real(
            int(m < 0) * np.sqrt(1 / 2) * (Y[(l, -m)] - 1j * Y[(l, m)])
            + int(m == 0) * Y[(l, 0)]
            + int(m > 0) * np.sqrt(1 / 2) * (Y[(l, m)] + 1j * Y[(l, -m)]) * (-1) ** m
    )
    for l in range(lmax + 1)
    for m in range(-l, l + 1)
}
Y = Y_real


def get_beta(data, ring_number, Y, lmax, smoothing=3, symmetrize=False):
    ring = data["rings"][ring_number]
    px, py, pz = ring
    if symmetrize:
        px, py, pz = np.vstack([px, -px]), np.vstack([py, py]), np.vstack([pz, -pz])
        px, py, pz = px.flatten(), py.flatten(), pz.flatten()

    pr = np.sqrt(px ** 2 + py ** 2 + pz ** 2)
    ptheta = np.arccos(py / pr)
    pphi = np.arctan2(pz, px)

    hist = np.histogram2d(pphi, ptheta, bins=(len(phiv), len(thetav)), density=True)[0]
    if smoothing > 0:
        hist = scipy.ndimage.gaussian_filter(hist, smoothing)
    beta = {
        (l, m): np.sum(hist * Y[(l, m)] * domega)
        for l in range(lmax + 1)
        for m in range(-l, l + 1)
    }
    return beta


betas = {
    (angle, ring): get_beta(datum, ring, Y, lmax)
    for angle, datum in zip(angles, data)
    for ring in range(len(datum["rings"]))
}

In [ ]:
plt.close("all")

# fig,ax=plt.subplots(lmax+1,2*lmax+1,figsize=(10,4))
# for l in range(lmax+1):
#     for m in range(-l,l+1):
#         ax[l,m+lmax].imshow(Y[(l,m)].T,cmap=cmr.fusion)
#         ax[l,m+lmax].set_title(f"l={l},m={m}")
#
# [axi.set_axis_off() for axi in ax.ravel() if axi]
# plt.tight_layout()

fig, ax = plt.subplots(2, 3, figsize=(10, 6))
fnum = 0
datum = data[fnum]
px, py, pz = datum["rings"][0]
px, py, pz = (
    np.vstack([px, -px]).flatten(),
    np.vstack([py, py]).flatten(),
    np.vstack([pz, -pz]).flatten(),
)
ax[0, 0].hist2d(px, py, bins=256)
ax[0, 1].hist2d(px, pz, bins=256)
ax[0, 2].hist2d(py, pz, bins=256)
px, py, pz = data[-(fnum + 1)]["rings"][0]
px, py, pz = (
    np.vstack([px, -px]).flatten(),
    np.vstack([py, py]).flatten(),
    np.vstack([pz, -pz]).flatten(),
)
ax[1, 0].hist2d(px, py, bins=256)
ax[1, 1].hist2d(px, pz, bins=256)
ax[1, 2].hist2d(py, pz, bins=256)
[axi.set_axis_off() for axi in ax.ravel() if axi]
plt.tight_layout()

fig, ax = plt.subplots(2, 3, figsize=(10, 6))
r = ((-0.5, 0.5), (-0.5, 0.5))
px, py, pz = data[fnum]["px"], data[fnum]["py"], data[fnum]["pz"]
ax[0, 0].hist2d(px, py, bins=256, range=r)
ax[0, 1].hist2d(px, pz, bins=256, range=r)
ax[0, 2].hist2d(py, pz, bins=256, range=r)
px, py, pz = data[-(fnum + 1)]["px"], data[-(fnum + 1)]["py"], data[-(fnum + 1)]["pz"]
ax[1, 0].hist2d(px, py, bins=256, range=r)
ax[1, 1].hist2d(px, pz, bins=256, range=r)
ax[1, 2].hist2d(py, pz, bins=256, range=r)
[axi.set_axis_off() for axi in ax.ravel() if axi]
plt.tight_layout()
plt.show()

In [ ]:
ring = 0
lm = 9
# angle_corr=[angle-10.5 for angle in angles]
angle_corr = angles
angle_lims = (-50, 50)
sampled_betas = [betas[(angle, ring)] for angle in angles]
plt.close("all")
plt.figure(figsize=(10, 6))
plt.subplot(221)
plt.title("Even l, Even m \n(Sym in plane, Sym Forwards/Backwards)")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(0,lm+1,2) for m in range(-l,l+1,2)]
[
    plt.plot(
            angle_corr,
            [b[(l, m)] + b[(l, -m)] for b in sampled_betas],
            label=f"l={l},|m|={m}",
    )
    for l in range(0, lm + 1, 2)
    for m in range(0, l + 1, 2)
]
plt.xlim(angle_lims)
plt.grid()
plt.legend()
plt.xlabel("Angle (deg)")
plt.subplot(222)
plt.title("Even l, Odd m \n(Antisym in plane, Antisym Forwards/Backwards)")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(0,lm+1,2) for m in range(-l+1,l,2)]
[
    plt.plot(
            angle_corr,
            [b[(l, m)] + b[(l, -m)] for b in sampled_betas],
            label=f"l={l},|m|={m}",
    )
    for l in range(0, lm + 1, 2)
    for m in range(1, l + 1, 2)
]
plt.xlim(angle_lims)
plt.grid()
plt.legend()
plt.xlabel("Angle (deg)")
plt.subplot(223)
plt.title("Odd l, Odd m \n(Antisym in plane, Sym Forwards/Backwards)")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(1,lm+1,2) for m in range(-l,l+1,2)]
[
    plt.plot(
            angle_corr,
            [b[(l, m)] + b[(l, -m)] for b in sampled_betas],
            label=f"l={l},|m|={m}",
    )
    for l in range(1, lm + 1, 2)
    for m in range(1, l + 1, 2)
]
plt.xlim(angle_lims)
plt.legend()
plt.xlabel("Angle (deg)")
plt.grid()
plt.subplot(224)
plt.title("Odd l, Even m \n(Sym in plane, Antisym Forwards/Backwards)")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(1,lm+1,2) for m in range(-l+1,l,2)]
[
    plt.plot(
            angle_corr,
            [b[(l, m)] + b[(l, -m)] for b in sampled_betas],
            label=f"l={l},|m|={m}",
    )
    for l in range(1, lm + 1, 2)
    for m in range(0, l + 1, 2)
]
plt.xlim(angle_lims)
plt.xlabel("Angle (deg)")
plt.grid()
plt.legend()
plt.suptitle(f"Beta Parameters, Ring {ring + 1}")
plt.tight_layout()
plt.figure(figsize=(10, 6))
plt.title("Odd l, m=0, Deviations from linear")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(1,lm+1,2) for m in range(-l+1,l,2)]
[
    plt.plot(
            angle_corr,
            [(b[(l, 0)] - sampled_betas[len(angles) // 2][(l, 0)]) for b in sampled_betas],
            label=f"l={l}, m=0, Total change={sampled_betas[-1][(l, 0)] - sampled_betas[0][(l, 0)]:.4f}",
    )
    for l in range(1, lm + 1, 2)
]
plt.xlim(angle_lims)
plt.xlabel("Angle (deg)")
plt.legend()
plt.show()

In [ ]:
sample, ring = 0, 0
PCD_Betas = {
    (l, m): betas[(angles[sample], ring)][(l, m)]
            - betas[(-angles[sample], ring)][(l, -m)]
    for l in range(-lmax, lmax + 1)
    for m in range(-l, l + 1)
}
PCD_PAD = sum(
        [
            PCD_Betas[(l, m)] * Y[(l, m)]
            for l in range(-lmax, lmax + 1)
            for m in range(-l, l + 1)
        ]
)
PCD_PAD_norm = (PCD_PAD - np.min(PCD_PAD)) / (np.max(PCD_PAD) - np.min(PCD_PAD))

Sum_Betas = {
    (l, m): betas[(angles[sample], ring)][(l, m)]
            + betas[(-angles[sample], ring)][(l, -m)]
    for l in range(-lmax, lmax + 1)
    for m in range(-l, l + 1)
}
Sum_PAD = sum(
        [
            Sum_Betas[(l, m)] * Y[(l, m)]
            for l in range(-lmax, lmax + 1)
            for m in range(-l, l + 1)
        ]
)
Sum_PAD_norm = (Sum_PAD - np.min(Sum_PAD)) / (np.max(Sum_PAD) - np.min(Sum_PAD))

r_ring = data[sample]["ring_vals"][ring]
x, y, z = (
    r_ring * np.sin(theta) * np.cos(phi),
    r_ring * np.sin(theta) * np.sin(phi),
    r_ring * np.cos(theta),
)
plt.close("all")
plt.figure(figsize=(10, 5))
ax = plt.subplot(121, projection="3d")
ax.plot_surface(x, y, z, rstride=8, cstride=8, facecolors=cmr.fusion(PCD_PAD_norm))
ax = plt.subplot(122, projection="3d")
ax.plot_surface(x, y, z, rstride=8, cstride=8, facecolors=cmr.ember(Sum_PAD_norm))

In [ ]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y, surfacecolor=PCD_PAD_norm)])
fig.show()